# Find proteins within the 8q gain event (cis) that are commonly equivalent between samples with and without the event

## Setup

In [1]:
import pandas as pd
import numpy as np
import os
import altair as alt

In [2]:
CHROMOSOME = "8"
ARM = "q"
TRANS_OR_CIS = "cis"

equiv_results_file = f"{CHROMOSOME}{ARM}_{TRANS_OR_CIS}_equiv.tsv"

equiv_results = pd.\
read_csv(equiv_results_file, sep="\t").\
rename(columns={"Name": "protein"}).\
set_index("protein")

In [3]:
equiv_results

,brca_Database_ID,luad_Database_ID,lscc_Database_ID,ovarian_Database_ID,brca_pvalue,colon_pvalue,hnscc_pvalue,lscc_pvalue,luad_pvalue,ovarian_pvalue
protein,,,,,,,,,,
ANKRD46,NP_001257308.1|NP_001257307.1,NP_001257308.1|NP_001257306.1,NP_001257308.1|NP_001257306.1,NP_001257307,0.176015,NaN,0.019119,0.130859,0.141650,0.140053
ASAP1,NP_060952.2|NP_001234925.1,NP_060952.2|NP_001234925.1|NP_001349854.1|NP_0...,NP_060952.2|NP_001234925.1|NP_001349854.1|NP_0...,NP_001234925,0.387188,0.021339,0.022845,0.012047,0.038956,0.140053
ATAD2,NP_054828.2,NP_054828.2|NP_001341036.1|NP_060022.2|NP_0012...,NP_054828.2,NP_054828,0.999802,0.090500,0.022726,0.487003,0.831469,0.024627
ATP6V1C1,NP_001686.1,NP_001686.1,NP_001686.1,NP_001686,0.789502,0.027222,0.022726,0.031567,0.040037,0.546696
C8orf37,NP_808880.1,NP_808880.1|NP_001350189.1,NP_808880.1|NP_001350189.1,NP_808880,0.132869,NaN,0.019119,0.452851,0.195066,0.088741
CA1,NP_001158302.1|NP_001278896.1,NP_001122301.1|NP_001278896.1,NP_001122301.1|NP_001278896.1,NP_001158302,0.124675,0.032493,0.027416,0.028722,0.043904,0.013286
CA1,NP_001158302.1|NP_001278896.1,NP_001278897.1,NP_001122301.1|NP_001278896.1,NP_001158302,0.124675,0.032493,0.027416,0.028722,0.055029,0.013286
CA1,NP_001158302.1|NP_001278896.1,NP_001122301.1|NP_001278896.1,NP_001278897.1,NP_001158302,0.124675,0.032493,0.027416,0.012047,0.043904,0.013286
CA1,NP_001158302.1|NP_001278896.1,NP_001278897.1,NP_001278897.1,NP_001158302,0.124675,0.032493,0.027416,0.012047,0.055029,0.013286


## Reshape the input dataframe
We want to get our table to have these columns:
- cancer_type
- protein
- Database_ID
- change
- p_value

Since some cancer types have database IDs and some don't, we'll slice out and reshape the info for each cancer type individually.

In [4]:
cancer_types = sorted(equiv_results.columns.to_series().str.split("_", n=1, expand=True)[0].unique())

long_results = pd.DataFrame()

for cancer_type in cancer_types:
    cancer_df = equiv_results.\
    loc[:, equiv_results.columns.str.startswith(cancer_type)].\
    dropna(axis="index", how="all").\
    reset_index(drop=False)
    
    # If the cancer type has database IDs, make a separate column that has them.
    # If not, create a column of NaNs (so that the tables all match)
    if f"{cancer_type}_Database_ID" in cancer_df.columns:
        cancer_df = cancer_df.rename(columns={f"{cancer_type}_Database_ID": "Database_ID"})
    else:
        cancer_df = cancer_df.assign(Database_ID=np.nan)
        
    # Rename the pvalue column to not have the cancer type
    cancer_df = cancer_df.\
    rename(columns={f"{cancer_type}_pvalue": "adj_p"}).\
    assign(cancer_type=cancer_type)
    
    # Reorder the columns
    cancer_df = cancer_df[["cancer_type", "protein", "Database_ID", "adj_p"]]
    
    # Append to the overall dataframe
    long_results = long_results.append(cancer_df)

# Drop duplicate rows and reset the index
long_results = long_results[~long_results.duplicated(keep=False)].\
reset_index(drop=True)

In [5]:
long_results

,cancer_type,protein,Database_ID,adj_p
0,brca,ANKRD46,NP_001257308.1|NP_001257307.1,0.176015
1,brca,ASAP1,NP_060952.2|NP_001234925.1,0.387188
2,brca,ATAD2,NP_054828.2,0.999802
3,brca,ATP6V1C1,NP_001686.1,0.789502
4,brca,C8orf37,NP_808880.1,0.132869
5,brca,CA13,NP_940986.1,0.067564
6,brca,CA3,NP_005172.1,0.103644
7,brca,CHMP4C,NP_689497.1,0.076885
8,brca,CNGB3,NP_061971.3,0.176035
9,brca,COL14A1,NP_066933.1,0.757354


## Select the equivalent proteins, and take a detour to make some plots

In [6]:
prots = long_results[long_results["adj_p"] <= 0.05].reset_index(drop=True)
prots_cts = prots.groupby("cancer_type").count()[["protein"]]

fail_prots = long_results[long_results["adj_p"] > 0.05].reset_index(drop=True)
fail_cts = fail_prots.groupby("cancer_type").count()[["protein"]]

prots_cts.insert(0, "count_type", "Equivalent")
fail_cts.insert(0, "count_type", "Not equivalent")

counts = prots_cts.append(fail_cts).sort_index().reset_index(drop=False)

alt.Chart(counts).mark_bar().encode(
    x=alt.X(
        "count_type",
        axis=alt.Axis(
            title=None,
            labels=False
        ),
        sort=["Equivalent"]
    ),
    y=alt.Y(
        "protein",
        axis=alt.Axis(
            title="Number of proteins"
        )
    ),
    color=alt.Color(
        "count_type",
        title=None,
        sort=["Equivalent"],
        scale=alt.Scale(
            domain=["Equivalent", "Not equivalent"],
            range=["#2d3da4", "#d1d1d1"]
        )
    )
).facet(
    column=alt.Column(
        "cancer_type",
        title=None
    )
).properties(
    title="Chr 8p cis equivalence"
).configure_title(
    anchor="middle"
)

alt.FacetChart(...)

## Find how many cancers each protein was different in

In [7]:
prots_summary = prots.groupby("protein").agg(**{
    "cancers": ("cancer_type", lambda x: x.sort_values().drop_duplicates(keep="first").tolist())
})

prots_summary = prots_summary.\
assign(
    num_cancers=prots_summary["cancers"].apply(len),
    tmp_sort=prots_summary["cancers"].apply(lambda x: "".join(x))
).\
sort_values(by=["num_cancers", "tmp_sort"], ascending=[False, True]).\
drop(columns="tmp_sort")

prots_summary

,cancers,num_cancers
protein,,
DEPTOR,"[brca, colon, hnscc, lscc, ovarian]",5
NDUFB9,"[brca, colon, lscc, luad, ovarian]",5
CA3,"[colon, hnscc, lscc, luad, ovarian]",5
RPL30,"[colon, hnscc, lscc, luad, ovarian]",5
IMPA1,"[brca, colon, hnscc, lscc]",4
INTS8,"[brca, colon, hnscc, ovarian]",4
NCALD,"[brca, colon, hnscc, ovarian]",4
TBC1D31,"[brca, hnscc, lscc, luad]",4
RALYL,"[brca, lscc, luad, ovarian]",4


## Check that there isn't overlap between proteins found as different and proteins found as equivalent

If there is overlap, we probably were too lenient in our bounds for the TOST test.

In [8]:
df = pd.read_csv(f"pancancer_summary_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}.tsv", sep="\t")
df = df.assign(cancers=df["cancers"].apply(lambda x: x.split("_")))

shared = df[df["protein"].isin(prots_summary.index)]
shared = shared.merge(
    right=prots_summary.reset_index(drop=False),
    on="protein",
    how="left", 
    suffixes=["_diff", "_equiv"],
)

ints = []
cts = []

for prot in shared["protein"]:
    diff_cancers = set(shared.loc[shared["protein"] == prot, "cancers_diff"].iloc[0])
    equiv_cancers = set(shared.loc[shared["protein"] == prot, "cancers_equiv"].iloc[0])
    
    intersection = diff_cancers.intersection(equiv_cancers)
    
    ints.append(intersection)
    cts.append(len(intersection))
    
shared = shared.assign(
    intersection=ints,
    intersection_size=cts
)

shared.sort_values(by="intersection_size", ascending=False)

,protein,cancers_diff,mean_simp_change,num_cancers_diff,cancers_equiv,num_cancers_equiv,intersection,intersection_size
0,SPAG1,"[brca, colon, hnscc, luad]",1,4,[lscc],1,{},0
25,WWP1,"[lscc, luad]",1,2,"[brca, colon, hnscc]",3,{},0
27,ZFAND1,"[luad, ovarian]",1,2,"[colon, hnscc]",2,{},0
28,ASAP1,[brca],1,1,"[colon, hnscc, lscc, luad]",4,{},0
29,COL14A1,[brca],-1,1,"[colon, ovarian]",2,{},0
30,CSMD3,[brca],1,1,[lscc],1,{},0
31,DPYS,[brca],-1,1,[lscc],1,{},0
32,ENPP2,[brca],-1,1,"[lscc, ovarian]",2,{},0
33,POLR2K,[brca],1,1,[lscc],1,{},0
34,RAD54B,[brca],1,1,"[hnscc, lscc]",2,{},0


## Out of curiousity, look at the ttest pvalues for difference, for the equivalent proteins

In [9]:
ttres = pd.\
read_csv(f"{CHROMOSOME}{ARM}_{TRANS_OR_CIS}effects_ttest.tsv", sep="\t").\
rename(columns={"Name": "protein"}).\
set_index("protein")

cancer_types = sorted(ttres.columns.to_series().str.split("_", n=1, expand=True)[0].unique())

ttres_long = pd.DataFrame()

for cancer_type in cancer_types:
    cancer_df = ttres.\
    loc[:, ttres.columns.str.startswith(cancer_type)].\
    dropna(axis="index", how="all").\
    reset_index(drop=False)
    
    # If the cancer type has database IDs, make a separate column that has them.
    # If not, create a column of NaNs (so that the tables all match)
    if f"{cancer_type}_Database_ID" in cancer_df.columns:
        cancer_df = cancer_df.rename(columns={f"{cancer_type}_Database_ID": "Database_ID"})
    else:
        cancer_df = cancer_df.assign(Database_ID=np.nan)
        
    # Rename the pvalue column to not have the cancer type
    cancer_df = cancer_df.\
    rename(columns={f"{cancer_type}_pvalue": "adj_p"}).\
    assign(cancer_type=cancer_type)
    
    # Reorder the columns
    cancer_df = cancer_df[["cancer_type", "protein", "Database_ID", "adj_p"]]
    
    # Append to the overall dataframe
    ttres_long = ttres_long.append(cancer_df)

# Drop duplicate rows and reset the index
ttres_long = ttres_long[~ttres_long.duplicated(keep=False)].\
reset_index(drop=True)

# Get the difference p values for all proteins that passed
diffs = pd.DataFrame()
for prot_idx in prots.index:
    
    prot = prots.loc[prot_idx, "protein"]
    cancer_type = prots.loc[prot_idx, "cancer_type"]
    
    diffs = diffs.append(ttres_long[
        (ttres_long["protein"] == prot) &
        (ttres_long["cancer_type"] == cancer_type)
    ])

# Plot the distribution of the difference p values for the equivalent proteins
alt.Chart(diffs).mark_bar().encode(
    x=alt.X(
        "adj_p",
        bin=alt.Bin(step=0.05),
        scale=alt.Scale(
            domain=[0, 1]
        )
    ),
    y=alt.Y(
        "count()"
    )
).properties(
    title="Distribution of difference p values for equivalent proteins"
)

alt.Chart(...)

In [10]:
# Plot the distribution of all the difference p values
alt.Chart(ttres_long).mark_bar().encode(
    x=alt.X(
        "adj_p",
        bin=alt.Bin(step=0.05),
        scale=alt.Scale(
            domain=[0, 1]
        )
    ),
    y=alt.Y(
        "count()"
    )
).properties(
    title="Distribution of difference p values for all proteins"
)

alt.Chart(...)

## Save results

In [11]:
prots_summary = prots_summary.assign(
    cancers=prots_summary["cancers"].apply(lambda x: "_".join(x))
)

output_file = f"pancancer_summary_equiv_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}.tsv"
prots_summary.to_csv(output_file, sep="\t")